# COSMOS Detection pipeline

This walkthrough shows how to perform basic document region detection over PDF documents using COSMOS. We'll use a trained model to classify regions on pages into tables, equations, etc

In [1]:
from dask.distributed import Client
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

## Setting up the database

We'll store our extraction results into SQLite. We've preinitialized an empty database at docs.db with the schema to be used. 

In [2]:
db_str = 'sqlite:///docs.db'
engine = create_engine(db_str)
engine

Engine(sqlite:///docs.db)

## Setting up the distributed backend

Cosmos uses Dask's lower level api, [Dask Distributed](https://distributed.dask.org/en/latest/) to handle its processing load. In this walkthrough, we'll setup a single node for processing, but this can be naturally extended to however many workers yo

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:43113 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 160 Memory: 540.25 GB


In [ ]:
ingested_objs_future = client.submit(ingest, obj, resources={'process': 1}, priority=-10)